In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
sns.set()

In [2]:
df = pd.read_csv('../data/data_merged.csv', dtype={'route':str})

In [3]:
df.shape

(144106, 11)

In [4]:
df.head(10)

,date,time,day,route,location,incident,delay,gap,direction,vehicle,incident id
0,2014-01-02,06:31:00,Thursday,505.0,Dundas and Roncesvalles,Late Leaving Garage,4.0,8.0,E/B,4018.0,NaN
1,2014-01-02,12:43:00,Thursday,504.0,King and Shaw,Utilized Off Route,20.0,22.0,E/B,4128.0,NaN
2,2014-01-02,14:01:00,Thursday,501.0,Kingston road and Bingham,Held By,13.0,19.0,W/B,4016.0,NaN
3,2014-01-02,14:22:00,Thursday,504.0,King St. and Roncesvalles Ave.,Investigation,7.0,11.0,W/B,4175.0,NaN
4,2014-01-02,16:42:00,Thursday,504.0,King and Bathurst,Utilized Off Route,3.0,6.0,E/B,4080.0,NaN
5,2014-01-02,17:39:00,Thursday,501.0,Queen and Beaconsfeild,Held By,7.0,13.0,W/B,4202.0,NaN
6,2014-01-02,18:38:00,Thursday,504.0,Roncesvalles and King Street West,Utilized Off Route,4.0,7.0,E/B,4100.0,NaN
7,2014-01-02,19:27:00,Thursday,510.0,Spadina and St. Andrews,Investigation,20.0,22.0,S/B,4123.0,NaN
8,2014-01-03,01:00:00,Friday,504.0,Broadview and Queen,Utilized Off Route,7.0,14.0,W/B,4079.0,NaN
9,2014-01-03,05:09:00,Friday,512.0,Bathurst and St. Clair,Mechanical,3.0,6.0,E/B,4160.0,NaN


In [5]:
df.dtypes

date            object
time            object
day             object
route           object
location        object
incident        object
delay          float64
gap            float64
direction       object
vehicle        float64
incident id    float64
dtype: object

In [6]:
df.isna().sum().sort_values()

date                0
time                0
day                 0
incident            1
delay              79
gap               121
location          264
route             417
vehicle          4685
direction       10355
incident id    143217
dtype: int64

## Investigate missing values

In [7]:
# 63/79 missing delays are covered
df[df['gap'].isna()].isna().sum().sort_values()

date             0
time             0
day              0
route            0
incident         0
location         3
direction        6
vehicle         17
delay           63
incident id    119
gap            121
dtype: int64

In [8]:
# 63/121 missing gaps are covered
df[df['delay'].isna()].isna().sum().sort_values()

date            0
time            0
day             0
route           0
incident        0
location        3
direction       5
vehicle        17
gap            63
delay          79
incident id    79
dtype: int64

### Drop missing values

In [9]:
# drop incident without reason
df = df[~df['incident'].isna()].copy()

In [10]:
# drop missing delays
df = df[~df['delay'].isna()].copy()
# drop missing gaps
df = df[~df['gap'].isna()].copy()

In [11]:
df.shape

(143968, 11)

### Drop irrelevant cols

In [12]:
df['incident id'].notna().sum()

887

In [13]:
df['incident id'].value_counts()

incident id
1.0     399
5.0     163
8.0     102
7.0      97
6.0      61
4.0      22
9.0      21
3.0      16
10.0      6
Name: count, dtype: int64

In [14]:
df.drop(columns={'incident id'},inplace=True)

## Standardize cols

### Create datetime for same-day analysis

In [15]:
df['date'] = pd.to_datetime(df['date'])

In [16]:
df['datetime'] = df['date'].astype(str)+" "+df['time']

In [17]:
df['datetime__len'] = df['datetime'].apply(len)

In [18]:
df['datetime__len'].value_counts()

datetime__len
19    82557
16    61405
30        6
Name: count, dtype: int64

In [19]:
df['dt'] = np.where(df['datetime__len']==19,df['datetime'],np.nan)

In [20]:
# add missing seconds
df['dt'] = np.where(df['datetime__len']==16,df['datetime']+':00',df['dt'])

In [21]:
# ignore duplicate dates
df['dt'] = np.where(df['datetime__len']==30,df['time'],df['dt'])

In [22]:
df['dt'] = pd.to_datetime(df['dt'])

In [23]:
df.drop(columns={'datetime','datetime__len'},inplace=True)

### Fix Vehicle

In [24]:
df['vehicle'] = df['vehicle'].apply(lambda x: str(x).replace('.0',''))

In [25]:
df['vehicle'] = np.where(df['vehicle']=='nan',np.nan,df['vehicle'])

### Fix Route

In [26]:
df['route'] = df['route'].apply(lambda x: str(x).replace('.0',''))

In [27]:
df['route'] = np.where(df['route']=='nan',np.nan,df['route'])

In [28]:
df[df['route'].isna()]['route'].value_counts()

Series([], Name: count, dtype: int64)

In [29]:
round(df['route'].value_counts(normalize=True)[:10]*100,1)

route
501    25.0
504    18.5
506    13.3
505    11.7
510     7.9
512     7.1
511     5.1
509     3.4
503     2.3
301     1.1
Name: proportion, dtype: float64

### Direction

In [30]:
df['direction'].nunique()

107

In [31]:
df['direction'] = df['direction'].str.lower()

In [32]:
df['direction'] = df['direction'].str.strip()

In [33]:
df['direction'].unique()

array(['e/b', 'w/b', 's/b', 'n/b', 'eb', 'wb', 'b/w', 'bw', 's', 'nb',
       'ew', 'eastbound', 'w', 'sb', 'southbound', 'northbound',
       'service adjusted.', 'westbound', nan, 'b#', 'we', 'e',
       'service adjusted', '14', '5', '0', 'sw', '2', '506', '54495',
       '4075', '12', 'bs', 'wruiter adv', '(ref', '9', 'gap', '1114',
       'run', 'ss', '19', '5                       w/b', '68029', 'eb``',
       '6', '512', 'w`', '\\5', '10', '4', '7', '13', '-bd#', '30', '`',
       '510', 'w.b', '\\', '26', 'w\\b', 'n', '20', '8', 'ee', '.',
       'see also', 'e5', 'relief operator. ga', '18', '15', 'e/b w/b',
       '33', 'e/w', 'eb/wb', 'n/s', 's/n', '-', '31087', 'nbn', 'wn', '3',
       'r', 'btw', 'b', 'c', 'q', 'u', 't', '1'], dtype=object)

In [34]:
from string import punctuation

# remove puncutation
df['direction'] = df['direction'].str.replace(rf'[{punctuation}]', '', regex=True)
df['direction'] = df['direction'].str.replace('\\','')

In [35]:
df['direction'] = df['direction'].str.replace('eastbound','eb').str.replace('westbound','wb').str.replace('southbound','sb').str.replace('northbound','nb')

In [36]:
df['direction'] = df['direction'].str.replace('bw','b')

In [37]:
pd.set_option('display.min_rows',300)

In [38]:
df['direction'] = df['direction'].str.replace(r'\d+','', regex=True)

In [39]:
df['direction__len'] = df['direction'].astype(str).apply(len)

In [40]:
# filter all directions greater than 2char
df['direction'] = np.where(df['direction__len']>2,np.nan,df['direction'])

In [41]:
df['direction'] = np.where(df['direction'].str.contains('nb|sb|eb|wb|n|s|e|w|ew|sw|ne|se|nw|se',regex=True),df['direction'],np.nan)

In [42]:
unpopular_directions = df['direction'].value_counts()[-9:].reset_index()['direction'].tolist()

In [43]:
df['direction'] = np.where(df['direction'].isin(unpopular_directions),np.nan,df['direction'])

In [44]:
df['direction'] = df['direction'].str.upper()

In [45]:
df['direction'].value_counts()

direction
WB    32768
EB    31993
W     19521
E     19346
S      6401
NB     6101
SB     5926
N      5891
Name: count, dtype: int64

In [46]:
def direction_simplifier(direction):
    if 'NB' in direction or 'NORTH' in direction or 'N\B' in direction or 'N' in direction:
        return 'N'
    elif 'SB' in direction or 'SOUTH' in direction or 'S\B' in direction or 'S' in direction:
        return 'S'
    elif 'EB' in direction or 'EAST' in direction or 'E\B' in direction or 'E' in direction:
        return 'E'
    elif 'WB' in direction or 'WEST' in direction or 'W\B' in direction or 'W' in direction:
        return 'W'
    elif 'BW' in direction or 'BWS' in direction or 'BOTH WAYS' in direction or 'BOTHWAY' in direction or 'BWAYS' in direction or 'B' in direction:
        return 'B'
    else:
        'NaN'

In [47]:
df['direction'] = df['direction'].astype(str)

In [48]:
df['direction'] = df['direction'].apply(direction_simplifier)

In [49]:
df.drop(columns={'direction__len'},inplace=True)

### Delay

In [50]:
df['delay'].describe()

count    143968.000000
mean         13.574218
std          32.561074
min           0.000000
25%           5.000000
50%           8.000000
75%          10.000000
max        1400.000000
Name: delay, dtype: float64

In [51]:
df[df['delay']==0].shape

(7128, 11)

### Gap

In [52]:
df['gap'].describe()

count    143968.000000
mean         19.963909
std          34.990974
min           0.000000
25%          10.000000
50%          15.000000
75%          20.000000
max        4216.000000
Name: gap, dtype: float64

In [53]:
df[df['gap']==0].shape

(7839, 11)

### Location

In [54]:
pd.set_option('display.min_rows',200)

In [55]:
df['location'] = df['location'].str.lower()

In [56]:
df['location'].nunique()

19183

In [57]:
df['location'] = df['location'].str.strip().str.replace(rf'[{punctuation}]', '', regex=True)

## Export

In [68]:
df.shape

(143968, 11)

In [67]:
df = df[['dt','date','time','day','route','location','incident','delay','gap','direction','vehicle']].copy()

In [69]:
df.to_csv('../data/data_cleaned.csv', index=None)